# Spanning trees

Anton Antonov   
July 2024

----

## Setup

In [ ]:
use Graph;

use Graph::Circulant;
use Graph::Complete;
use Graph::CompleteKaryTree;
use Graph::Cycle;
use Graph::Grid;
use Graph::Hypercube;
use Graph::KnightTour;
use Graph::Star;
use Graph::Path;
use Graph::Petersen;
use Graph::Wheel;

use Graph::Distribution;
use Graph::Random;

use Data::Reshapers;
use Data::Summarizers;
use Data::TypeSystem;
use Data::Translators;
use Data::Geographics;
use Math::DistanceFunctions;
use Math::Nearest;
use Hash::Merge;

use JavaScript::D3;
use WWW::MermaidInk;

In [ ]:
#% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

In [ ]:
#% js
js-d3-list-line-plot(10.rand xx 40, background => 'none', stroke-width => 2)

In [ ]:
my $title-color = 'Silver';
my $stroke-color = 'SlateGray';
my $tooltip-color = 'LightBlue';
my $tooltip-background-color = 'none';
my $background = '1F1F1F';
my $color-scheme = 'schemeTableau10';
my $edge-thickness = 3;
my $vertex-size = 6;
my $mmd-theme = q:to/END/;
%%{
  init: {
    'theme': 'forest',
    'themeVariables': {
      'lineColor': 'Ivory'
    }
  }
}%%
END
my %force = collision => {iterations => 0, radius => 10},link => {distance => 180};
my %force2 = charge => {strength => -30, iterations => 4}, collision => {radius => 50, iterations => 4}, link => {distance => 30};

my %opts = :$background, :$title-color, :$edge-thickness, :$vertex-size;

-----

## Small Geo-points set

A company is planning a fiber network for a number of Chicago suburbs. It only has the right of way for its fiber along certain corridors. Some of those corridors might be more expensive. Find the subgraph of connection corridors that connect every suburb with the lowest total cost:

In [ ]:
my @dsEdges =
        %('from'=>'Maywood','to'=>'Downers','weight'=>14),
        %('from'=>'Maywood','to'=>'Oakbrook','weight'=>28),
        %('from'=>'Maywood','to'=>'Addison','weight'=>10),
        %('from'=>'Downers','to'=>'Grove','weight'=>25),
        %('from'=>'Downers','to'=>'Stickney','weight'=>23),
        %('from'=>'Downers','to'=>'Wheaton','weight'=>20),
        %('from'=>'Downers','to'=>'Oakbrook','weight'=>27),
        %('from'=>'Oakbrook','to'=>'Stickney','weight'=>11),
        %('from'=>'Oakbrook','to'=>'Addison','weight'=>17),
        %('from'=>'Addison','to'=>'McCook','weight'=>21),
        %('from'=>'Grove','to'=>'Wheaton','weight'=>10),
        %('from'=>'Stickney','to'=>'Wheaton','weight'=>15),
        %('from'=>'Stickney','to'=>'Bellwood','weight'=>20),
        %('from'=>'Wheaton','to'=>'Bellwood','weight'=>10),
        %('from'=>'Bellwood','to'=>'McCook','weight'=>28);

deduce-type(@dsEdges)

In [ ]:
my $g = Graph.new(@dsEdges)

In [ ]:
#% js
$g.edges(:dataset)
==> js-d3-graph-plot(
    width => 800,
    :$background, :$title-color, :$edge-thickness, :$vertex-size,
    force => {charge => { strength => -800}, collision => {radius => 50}}
    )

In [ ]:
my $stree = $g.find-spanning-tree;

In [ ]:
#% js
$g.edges
==> js-d3-graph-plot(
    highlight => $stree.edges,
    width => 800,
    :$background, :$title-color, :$edge-thickness, :$vertex-size,
    force => {charge => { strength => -800}, collision => {radius => 50}}
    )

-------

## Grid graph

In [ ]:
my $g = Graph::Grid.new(n => 4, m => 6)

In [ ]:
my $stree = $g.find-spanning-tree

In [ ]:
#% js
$g.edges
==> js-d3-graph-plot(
    highlight => [|$stree.vertex-list, |$stree.edges],
    width => 500,
    :$background, :$title-color, :$edge-thickness, :$vertex-size,
    force => {charge => {strength => -160, iterations => 2}, collision => {radius => 4, iterations => 1}, link => {distance => 0}}
    )

-----

## Florida cities

Build an interstate highway system joining the cities of the state of Florida, USA.

Here is a dataset of large enough cities:

In [ ]:
my @cities = city-data().grep({ $_<State> eq 'Florida' && $_<Population> ≥ 50_000 });
@cities.elems

In [ ]:
#% html
@cities.head(4) ==> to-html

Here is the corresponding weighted graph:

In [ ]:
my %coords = @cities.map({ $_<City> => ($_<Latitude>, $_<Longitude>) });
my @dsEdges = (%coords.keys X %coords.keys ).map({ %(from => $_.head, to => $_.tail, weight => geo-distance(|%coords{$_.head}, |%coords{$_.tail} )) })

In [ ]:
my $g = Graph.new(@dsEdges, :!directed)

In [ ]:
my $stree = $g.find-spanning-tree(method => 'kruskal')

In [ ]:
#my $stree2 = $g.find-spanning-tree(method => 'prim')

In [ ]:
%coords.nodemap({.reverse}).Hash

In [ ]:
#% js
$stree.edges
==> js-d3-graph-plot(
    vertex-coordinates => %coords.nodemap(*.reverse)».List.Hash,
    width => 700,
    height => 1000,
    :$background, :$title-color, :$edge-thickness, 
    vertex-size => 4,
    vertex-label-font-size => 9,
    margins => {right => 200}
    )

--------

## Africa routes

In [ ]:
my @africaEdges = %('from'=>'Algeria','to'=>'Libya','weight'=>1),%('from'=>'Algeria','to'=>'Mali','weight'=>1),%('from'=>'Algeria','to'=>'Mauritania','weight'=>1),%('from'=>'Algeria','to'=>'Morocco','weight'=>1),%('from'=>'Algeria','to'=>'Niger','weight'=>1),%('from'=>'Algeria','to'=>'Tunisia','weight'=>1),%('from'=>'Algeria','to'=>'WesternSahara','weight'=>1),%('from'=>'Libya','to'=>'Niger','weight'=>1),%('from'=>'Libya','to'=>'Tunisia','weight'=>1),%('from'=>'Libya','to'=>'Chad','weight'=>1),%('from'=>'Libya','to'=>'Egypt','weight'=>1),%('from'=>'Libya','to'=>'Sudan','weight'=>1),%('from'=>'Mali','to'=>'Mauritania','weight'=>1),%('from'=>'Mali','to'=>'Niger','weight'=>1),%('from'=>'Mali','to'=>'BurkinaFaso','weight'=>1),%('from'=>'Mali','to'=>'Guinea','weight'=>1),%('from'=>'Mali','to'=>'IvoryCoast','weight'=>1),%('from'=>'Mali','to'=>'Senegal','weight'=>1),%('from'=>'Mauritania','to'=>'WesternSahara','weight'=>1),%('from'=>'Mauritania','to'=>'Senegal','weight'=>1),%('from'=>'Morocco','to'=>'WesternSahara','weight'=>1),%('from'=>'Niger','to'=>'Chad','weight'=>1),%('from'=>'Niger','to'=>'BurkinaFaso','weight'=>1),%('from'=>'Niger','to'=>'Benin','weight'=>1),%('from'=>'Niger','to'=>'Nigeria','weight'=>1),%('from'=>'Chad','to'=>'Sudan','weight'=>1),%('from'=>'Chad','to'=>'Nigeria','weight'=>1),%('from'=>'Chad','to'=>'Cameroon','weight'=>1),%('from'=>'Chad','to'=>'CentralAfricanRepublic','weight'=>1),%('from'=>'Egypt','to'=>'Sudan','weight'=>1),%('from'=>'Sudan','to'=>'CentralAfricanRepublic','weight'=>1),%('from'=>'Sudan','to'=>'Eritrea','weight'=>1),%('from'=>'Sudan','to'=>'Ethiopia','weight'=>1),%('from'=>'Sudan','to'=>'SouthSudan','weight'=>1),%('from'=>'BurkinaFaso','to'=>'IvoryCoast','weight'=>1),%('from'=>'BurkinaFaso','to'=>'Benin','weight'=>1),%('from'=>'BurkinaFaso','to'=>'Ghana','weight'=>1),%('from'=>'BurkinaFaso','to'=>'Togo','weight'=>1),%('from'=>'Guinea','to'=>'IvoryCoast','weight'=>1),%('from'=>'Guinea','to'=>'Senegal','weight'=>1),%('from'=>'Guinea','to'=>'GuineaBissau','weight'=>1),%('from'=>'Guinea','to'=>'Liberia','weight'=>1),%('from'=>'Guinea','to'=>'SierraLeone','weight'=>1),%('from'=>'IvoryCoast','to'=>'Ghana','weight'=>1),%('from'=>'IvoryCoast','to'=>'Liberia','weight'=>1),%('from'=>'Senegal','to'=>'GuineaBissau','weight'=>1),%('from'=>'Senegal','to'=>'Gambia','weight'=>1),%('from'=>'Benin','to'=>'Nigeria','weight'=>1),%('from'=>'Benin','to'=>'Togo','weight'=>1),%('from'=>'Nigeria','to'=>'Cameroon','weight'=>1),%('from'=>'Cameroon','to'=>'CentralAfricanRepublic','weight'=>1),%('from'=>'Cameroon','to'=>'EquatorialGuinea','weight'=>1),%('from'=>'Cameroon','to'=>'Gabon','weight'=>1),%('from'=>'Cameroon','to'=>'RepublicCongo','weight'=>1),%('from'=>'CentralAfricanRepublic','to'=>'SouthSudan','weight'=>1),%('from'=>'CentralAfricanRepublic','to'=>'RepublicCongo','weight'=>1),%('from'=>'CentralAfricanRepublic','to'=>'DemocraticRepublicCongo','weight'=>1),%('from'=>'Eritrea','to'=>'Ethiopia','weight'=>1),%('from'=>'Eritrea','to'=>'SouthSudan','weight'=>1),%('from'=>'Eritrea','to'=>'Djibouti','weight'=>1),%('from'=>'Ethiopia','to'=>'SouthSudan','weight'=>1),%('from'=>'Ethiopia','to'=>'Djibouti','weight'=>1),%('from'=>'Ethiopia','to'=>'Kenya','weight'=>1),%('from'=>'Ethiopia','to'=>'Somalia','weight'=>1),%('from'=>'SouthSudan','to'=>'DemocraticRepublicCongo','weight'=>1),%('from'=>'SouthSudan','to'=>'Kenya','weight'=>1),%('from'=>'SouthSudan','to'=>'Uganda','weight'=>1),%('from'=>'Ghana','to'=>'Togo','weight'=>1),%('from'=>'Liberia','to'=>'SierraLeone','weight'=>1),%('from'=>'EquatorialGuinea','to'=>'Gabon','weight'=>1),%('from'=>'Gabon','to'=>'RepublicCongo','weight'=>1),%('from'=>'RepublicCongo','to'=>'DemocraticRepublicCongo','weight'=>1),%('from'=>'RepublicCongo','to'=>'Angola','weight'=>1),%('from'=>'DemocraticRepublicCongo','to'=>'Uganda','weight'=>1),%('from'=>'DemocraticRepublicCongo','to'=>'Angola','weight'=>1),%('from'=>'DemocraticRepublicCongo','to'=>'Burundi','weight'=>1),%('from'=>'DemocraticRepublicCongo','to'=>'Rwanda','weight'=>1),%('from'=>'DemocraticRepublicCongo','to'=>'Tanzania','weight'=>1),%('from'=>'DemocraticRepublicCongo','to'=>'Zambia','weight'=>1),%('from'=>'Djibouti','to'=>'Somalia','weight'=>1),%('from'=>'Kenya','to'=>'Somalia','weight'=>1),%('from'=>'Kenya','to'=>'Uganda','weight'=>1),%('from'=>'Kenya','to'=>'Tanzania','weight'=>1),%('from'=>'Uganda','to'=>'Rwanda','weight'=>1),%('from'=>'Uganda','to'=>'Tanzania','weight'=>1),%('from'=>'Angola','to'=>'Zambia','weight'=>1),%('from'=>'Angola','to'=>'Namibia','weight'=>1),%('from'=>'Burundi','to'=>'Rwanda','weight'=>1),%('from'=>'Burundi','to'=>'Tanzania','weight'=>1),%('from'=>'Rwanda','to'=>'Tanzania','weight'=>1),%('from'=>'Tanzania','to'=>'Zambia','weight'=>1),%('from'=>'Tanzania','to'=>'Malawi','weight'=>1),%('from'=>'Tanzania','to'=>'Mozambique','weight'=>1),%('from'=>'Zambia','to'=>'Namibia','weight'=>1),%('from'=>'Zambia','to'=>'Malawi','weight'=>1),%('from'=>'Zambia','to'=>'Mozambique','weight'=>1),%('from'=>'Zambia','to'=>'Botswana','weight'=>1),%('from'=>'Zambia','to'=>'Zimbabwe','weight'=>1),%('from'=>'Namibia','to'=>'Botswana','weight'=>1),%('from'=>'Namibia','to'=>'SouthAfrica','weight'=>1),%('from'=>'Malawi','to'=>'Mozambique','weight'=>1),%('from'=>'Mozambique','to'=>'Zimbabwe','weight'=>1),%('from'=>'Mozambique','to'=>'SouthAfrica','weight'=>1),%('from'=>'Mozambique','to'=>'Swaziland','weight'=>1),%('from'=>'Botswana','to'=>'Zimbabwe','weight'=>1),%('from'=>'Botswana','to'=>'SouthAfrica','weight'=>1),%('from'=>'Zimbabwe','to'=>'SouthAfrica','weight'=>1),%('from'=>'SouthAfrica','to'=>'Swaziland','weight'=>1),%('from'=>'SouthAfrica','to'=>'Lesotho','weight'=>1);

deduce-type(@africaEdges)

In [ ]:
sink records-summary(@africaEdges)

In [ ]:
my %africaCoords = 'Algeria'=>[3.0,28.0],'Libya'=>[17.0,25.0],'Mali'=>[-4.0,17.0],'Mauritania'=>[-12.0,20.0],'Morocco'=>[-5.0,32.0],'Niger'=>[8.0,16.0],'Tunisia'=>[9.0,34.0],'WesternSahara'=>[-13.0,24.5],'Chad'=>[19.0,15.0],'Egypt'=>[30.0,27.0],'Sudan'=>[30.0,13.8],'BurkinaFaso'=>[-2.0,13.0],'Guinea'=>[-10.0,11.0],'IvoryCoast'=>[-5.0,8.0],'Senegal'=>[-14.0,14.0],'Benin'=>[2.25,9.5],'Nigeria'=>[8.0,10.0],'Cameroon'=>[12.0,6.0],'CentralAfricanRepublic'=>[21.0,7.0],'Eritrea'=>[39.0,15.0],'Ethiopia'=>[38.0,8.0],'SouthSudan'=>[30.51,6.7],'Ghana'=>[-2.0,8.0],'Togo'=>[1.1667,8.0],'GuineaBissau'=>[-15.0,12.0],'Liberia'=>[-9.5,6.5],'SierraLeone'=>[-11.5,8.5],'Gambia'=>[-16.5667,13.4667],'EquatorialGuinea'=>[10.0,2.0],'Gabon'=>[11.75,-1.0],'RepublicCongo'=>[15.0,-1.0],'DemocraticRepublicCongo'=>[25.0,0.0],'Djibouti'=>[43.0,11.5],'Kenya'=>[38.0,1.0],'Somalia'=>[49.0,10.0],'Uganda'=>[32.0,1.0],'Angola'=>[18.5,-12.5],'Burundi'=>[30.0,-3.5],'Rwanda'=>[30.0,-2.0],'Tanzania'=>[35.0,-6.0],'Zambia'=>[30.0,-15.0],'Namibia'=>[17.0,-22.0],'Malawi'=>[34.0,-13.5],'Mozambique'=>[35.0,-18.25],'Botswana'=>[24.0,-22.0],'Zimbabwe'=>[30.0,-20.0],'SouthAfrica'=>[24.0,-29.0],'Swaziland'=>[31.5,-26.5],'Lesotho'=>[28.5,-29.5];
%africaCoords .= deepmap(*.Numeric);
deduce-type(%africaCoords)

In [ ]:
sink records-summary(%africaCoords.values)

In [ ]:
my $africaGraph = Graph.new(@africaEdges);

In [ ]:
my $africanTree = $africaGraph.find-spanning-tree

In [ ]:
#% js
js-d3-graph-plot(
    $africaGraph.edges,
    highlight => [|$africanTree.vertex-list, |$africanTree.edges],
    vertex-coordinates => %africaCoords,
    highlight-color => 'Blue',
    width => 600,
    height => 600,
    vertex-label-color => 'none',
    :$background, :$title-color, :$edge-thickness, :$vertex-size,
    )

-------

## Graph by list-line-plot

In [ ]:
my $k = 0;
my @africaEdges2 = @africaEdges.map({ 
    [
        %(x => %africaCoords{$_<from>}.head, y => %africaCoords{$_<from>}.tail, group => ~$k), 
        %(x => %africaCoords{$_<to>}.head, y => %africaCoords{$_<to>}.tail, group => ~$k++), 
    ]}).flat

In [ ]:
#%js
js-d3-list-line-plot(@africaEdges2,
    width => 600,
    height => 600,
    :grid-lines,
    :$background, 
    color-scheme => 'schemePaired',
    stroke-width => 3,
    :!legends
).subst('.range(d3.schemePaired);', '.range(["SteelBlue"]);')